In [10]:
from DatasetTools import loadDataset,Datasets,findCommonFeatures,combineDatasets
from Loader.CombinedDataset import CombinationMethod
import pandas as pd
import numpy as np
import os
import time
cwd = os.getcwd()

calcFeatures = True
usb1 = loadDataset(Datasets.USBIDS_2021,'../datasets/USBIDS21/Slowhttptest-NoDefense.csv',"USB-IDS Slowhttptest",calcFeatures=calcFeatures)
usb2 = loadDataset(Datasets.USBIDS_2021,'../datasets/USBIDS21/REGULAR.csv',"USB-IDS Regular",calcFeatures=calcFeatures)

UNSW1 = loadDataset(Datasets.UNSW_NB15_2015,"../datasets/UNSW-NB15/UNSW-NB15_1.csv","UNSW-NB15_1",calcFeatures=calcFeatures)
UNSW2 = loadDataset(Datasets.UNSW_NB15_2015,"../datasets/UNSW-NB15/UNSW-NB15_2.csv","UNSW-NB15_2",calcFeatures=calcFeatures)

loading Datasets.USBIDS_2021 from "../datasets/USBIDS21/Slowhttptest-NoDefense.csv" with dataset name: USB-IDS Slowhttptest
loading Datasets.USBIDS_2021 from "../datasets/USBIDS21/REGULAR.csv" with dataset name: USB-IDS Regular
loading Datasets.UNSW_NB15_2015 from "../datasets/UNSW-NB15/UNSW-NB15_1.csv" with dataset name: UNSW-NB15_1
loading Datasets.UNSW_NB15_2015 from "../datasets/UNSW-NB15/UNSW-NB15_2.csv" with dataset name: UNSW-NB15_2


In [9]:
print("USB1")
timeinit = time.time()
tmp1 = next(usb1)
print(f'TIME::TotalProc={(timeinit-time.time())*1000.0}ms') #! timing test

print("USB2")
timeinit = time.time()
tmp2 = next(usb2)
print(f'TIME::TotalProc={(timeinit-time.time())*1000.0}ms') #! timing test

print("UNSW1")
timeinit = time.time()
tmp3 = next(UNSW1)
print(f'TIME::TotalProc={(timeinit-time.time())*1000.0}ms') #! timing test

print("UNSW2")
timeinit = time.time()
tmp4 = next(UNSW2)
print(f'TIME::TotalProc={(timeinit-time.time())*1000.0}ms') #! timing test

USB1
TIME::TotalProc=-301.5463352203369ms
USB2
TIME::TotalProc=-5130.762100219727ms
UNSW1


/home/matthew/2022SummerResearch/DatasetProcessor/Loader/Dataset.py:145: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = next(self.reader)


TIME::TotalProc=-1020.4427242279053ms
UNSW2
TIME::TotalProc=-974.1029739379883ms


In [ ]:
print(tmp1.Protocol.unique())
print(tmp2.Protocol.unique())
print(tmp3.Protocol.unique())
print(tmp4.Protocol.unique())

In [ ]:
import itertools

proto = set()
for t in itertools.chain(UNSW1,UNSW2):
    for p in t.Protocol.unique():
        proto.add(p)
print(proto)

In [1]:
items = findCommonFeatures(usb1,usb2,UNSW1,UNSW2)
usbItems = usb1.getFeatures()
unswItems = UNSW1.getFeatures()
missingUSB = [v for v in usbItems if v not in items]
missingUNSW = [v for v in unswItems if v not in items]

print(f"Combined has {len(items)} features")
print(f"USBIDS has {len(usbItems)} features, {len(missingUSB)} are missing")
print(f"UNSWNB15 has {len(unswItems)} features, {len(missingUNSW)} are missing")


NameError: name 'findCommonFeatures' is not defined

In [11]:
comb = combineDatasets(usb1, usb2, UNSW1,UNSW2, method = CombinationMethod.INTERLACE)
count = 0

In [12]:
comb.chunksize = 10 ** 6

In [13]:
#first boundary
temp = next(comb)
count=1
temp.memory_usage().sum()/1024

/home/matthew/2022SummerResearch/DatasetProcessor/Loader/Dataset.py:146: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = self.reader.get_chunk(rows)


242187.5

In [5]:
#second boundary
for i in range(3):
    temp = next(comb)
    count+=1

Combining USB-IDS Regular
Combining USB-IDS Regular
Reached end of USB-IDS Regular with 12658 flows, switching to UNSW-NB15_1


/home/matthew/2022SummerResearch/DatasetProcessor/Loader/Dataset.py:136: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = self.reader.get_chunk(rows)


Combining UNSW-NB15_1


/home/matthew/2022SummerResearch/DatasetProcessor/Loader/Dataset.py:136: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = self.reader.get_chunk(rows)


In [ ]:
np.all(temp["Dataset"] == "USB-IDS Regular")

In [ ]:
print(items)

In [ ]:
#! OLD
comb1 = combineDatasets(usb1, usb2, method = CombinationMethod.SEQUENTIAL)
items.append("Dataset")
df = next(comb1)[items]
for d in comb1:
    df = pd.concat([df,d[items]])

print(f'{len(df)} features')
stats1 = df.describe()
sample1 = df.sample(n=5)

In [ ]:
stime = comb1.datasetTimes[-1][1]

In [ ]:
#! OLD
comb2 = combineDatasets(UNSW1, UNSW2, method = CombinationMethod.SEQUENTIAL,startTime = stime)
df = next(comb2)[items]
for d in comb2:
    df = pd.concat([df,d[items]])

print(f'{len(df)} features')
stats2 = df.describe()
sample2 = df.sample(n=5)

In [ ]:
sample2 = df.sample(n=5)

In [ ]:
sample=pd.concat([sample1,sample2])
sample.to_csv("../datasets/sample.csv")

In [2]:
comb1 = combineDatasets(usb1, usb2, method = CombinationMethod.SEQUENTIAL)
comb2 = combineDatasets(UNSW1, UNSW2, method = CombinationMethod.SEQUENTIAL)
comb = combineDatasets(comb1, comb2, method = CombinationMethod.SEQUENTIAL)

In [3]:
temp = next(comb)

Combining Combined
Combining USB-IDS Slowhttptest
Reached end of USB-IDS Slowhttptest with 6737 flows, switching to USB-IDS Regular
